# Proyecto 1
## Detección de ataques

In [43]:
import os.path

import pandas as pd
from pandas_profiling import ProfileReport

import utils

In [44]:
generate_reports: bool = True
generate_dfs: bool = True

In [45]:
df = pd.read_csv("dataset/sampled_df.csv")

df.shape

(488370, 50)

## Preprocessing

### Remove unnecesary variables

#### Drop constants

In [46]:
# CONSTANTS 
constants = [
    'BIFLOW_DIRECTION',
    'FIREWALL_EVENT',
    'FLOW_ACTIVE_TIMEOUT',
    'FLOW_INACTIVE_TIMEOUT',
    'FRAME_LENGTH',
    'MAX_IP_PKT_LEN',
    'MIN_IP_PKT_LEN',
    'SAMPLING_INTERVAL',
    'DIRECTION',
    'OOORDER_IN_PKTS',
    'OOORDER_OUT_PKTS',
]

# DIRECTION, OOORDER_IN_PKTS and OOORDER_OUT_PKTS has 99%> zero values, so it will be considered a constant.

In [47]:
# Drop constants from dataframe
df = df.drop(constants, axis=1)
len(df.columns.values)

39

#### Drop unique values

In [48]:
# UNIQUE
unique_vars = ['FLOW_ID', 'TOTAL_FLOWS_EXP']

In [49]:
df = df.drop(unique_vars, axis=1)
len(df.columns.values)

37

#### DROP IPS

In [50]:
df = df.drop(["IPV4_DST_ADDR", "IPV4_SRC_ADDR"], axis = 1)

#### DROP PROTOCOL COLUMN

In [51]:
df = df.drop(["PROTOCOL"], axis = 1)

#### Clean data from report

In [52]:
# Drop columns that almost everything is 0
df = df.drop(columns=["RETRANSMITTED_IN_BYTES", "RETRANSMITTED_IN_PKTS"], axis = 1)

In [53]:
df = df.drop(columns=["RETRANSMITTED_OUT_BYTES", "RETRANSMITTED_OUT_PKTS"], axis=1)

In [54]:
df = df.drop(columns=["TCP_WIN_MSS_OUT", "TCP_WIN_SCALE_IN", "TCP_WIN_SCALE_IN", "TCP_WIN_SCALE_OUT", "SRC_TOS", "DST_TOS"], axis=1)

### Numerical preprocessing

#### Fix DST_TO_SRC_SECOND_BYTES column

In [55]:
# df["DST_TO_SRC_SECOND_BYTES"].apply(np.isreal)
DST_TO_SRC_SECOND_BYTES_INT_VALUES = list(filter(utils.is_int, df["DST_TO_SRC_SECOND_BYTES"]))
DST_TO_SRC_SECOND_BYTES_NON_INT_VALUES = list(filter(lambda x: not utils.is_int(x), df["DST_TO_SRC_SECOND_BYTES"]))

In [56]:
print("Correct int values: ", len(DST_TO_SRC_SECOND_BYTES_INT_VALUES))
print("Incorrect int values: ", len(DST_TO_SRC_SECOND_BYTES_NON_INT_VALUES))
print("Total values count: ", len(df["DST_TO_SRC_SECOND_BYTES"]))

Correct int values:  184707
Incorrect int values:  303663
Total values count:  488370


In [57]:
df_safe_clean = df.copy()

In [58]:
df["DST_TO_SRC_SECOND_BYTES"] = df["DST_TO_SRC_SECOND_BYTES"].apply(lambda x: int(x) if utils.is_int(x) else utils.normalize_dst_to_src_column(x))
df["DST_TO_SRC_SECOND_BYTES"].head()

3369345    624
4767061    156
2811993    736
3673849    624
4013181    624
Name: DST_TO_SRC_SECOND_BYTES, dtype: int64

In [59]:
# Replace the 0 with 1
df["DST_TO_SRC_SECOND_BYTES"].replace({0: 1}, inplace=True)
df["DST_TO_SRC_SECOND_BYTES"].head()

3369345    624
4767061    156
2811993    736
3673849    624
4013181    624
Name: DST_TO_SRC_SECOND_BYTES, dtype: int64

#### Fix SRC_TO_DST_SECOND_BYTES column

In [60]:
df["SRC_TO_DST_SECOND_BYTES"] = df["SRC_TO_DST_SECOND_BYTES"].apply(lambda x: int(x) if utils.is_int(x) else utils.normalize_dst_to_src_column(x))
df["SRC_TO_DST_SECOND_BYTES"].head()

3369345    1275
4767061     349
2811993    1563
3673849    1329
4013181    1155
Name: SRC_TO_DST_SECOND_BYTES, dtype: int64

### Second profile report

In [61]:
if not os.path.isfile("reports/profile-numerical.html") and generate_reports:
    ProfileReport(df, title="Profile after numerical preprocessing", minimal=True).to_file("reports/profile-numerical.html")

#### Reduce skew data

In [62]:
import numpy as np
from scipy.stats import skew

def reduce_skew(column, fn = np.log):
    print("Skew actual value for column ", column,": ",skew(df["SRC_TO_DST_SECOND_BYTES"]))
    df[column].replace({0: 1}, inplace=True)
    df[column] = df[column].apply(fn)
    print("New value: ", skew(df[column]))

In [63]:
reduce_skew("SRC_TO_DST_SECOND_BYTES")
reduce_skew("DST_TO_SRC_SECOND_BYTES")
reduce_skew("OUT_BYTES")

Skew actual value for column  SRC_TO_DST_SECOND_BYTES :  319.2781970169553
New value:  0.214632391445567
Skew actual value for column  DST_TO_SRC_SECOND_BYTES :  0.214632391445567
New value:  -0.5959331050626652
Skew actual value for column  OUT_BYTES :  0.214632391445567
New value:  -0.5950657677745992


In [64]:
reduce_skew("IN_BYTES")
reduce_skew("IN_PKTS")
reduce_skew("OUT_PKTS")

Skew actual value for column  IN_BYTES :  0.214632391445567
New value:  0.21731618581930975
Skew actual value for column  IN_PKTS :  0.214632391445567
New value:  0.2805405454915055
Skew actual value for column  OUT_PKTS :  0.214632391445567
New value:  0.4811800921206545


### Correlation check

In [65]:
import seaborn as sn
import matplotlib.pyplot as plt
corrMatrix = df.corr()

In [66]:
fig, ax = plt.subplots(figsize=(60,20)) 
sn.heatmap(corrMatrix, annot=True, linewidths=.5, ax=ax)
plt.show()

/Users/douglasdeleon/Documents/DouglasJr/UVG/semestre9/SecurityDS/proyecto-1/attack-detection/venv/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  This is separate from the ipykernel package so we can avoid doing imports until


In [67]:
for column in corrMatrix:
        correl = []
        for index in corrMatrix[column].index:
            if column != index and abs(corrMatrix[column][index]) >= 0.75:
                correl.append((index, corrMatrix[column][index]))
        if len(correl) > 0:
            print("La columna ", column, " tuvo correlación con las columnas")
            print(correl)
            print("\n")

La columna  DST_TO_SRC_SECOND_BYTES  tuvo correlación con las columnas
[('IN_BYTES', 0.7750787092436623), ('IN_PKTS', 0.7868472677192258), ('OUT_BYTES', 0.9999990908664482), ('OUT_PKTS', 0.8145579752781831), ('SRC_TO_DST_SECOND_BYTES', 0.7751061109748211)]


La columna  FIRST_SWITCHED  tuvo correlación con las columnas
[('FLOW_END_MILLISECONDS', 0.9999999987112844), ('FLOW_END_SEC', 0.999999998712298), ('FLOW_START_MILLISECONDS', 0.9999999999999297), ('FLOW_START_SEC', 1.0), ('LAST_SWITCHED', 0.999999998712298)]


La columna  FLOW_DURATION_MICROSECONDS  tuvo correlación con las columnas
[('FLOW_DURATION_MILLISECONDS', 0.999999999972291), ('IN_BYTES', 0.765416536990902), ('IN_PKTS', 0.7951984216949622), ('OUT_PKTS', 0.7876313170396291), ('SRC_TO_DST_SECOND_BYTES', 0.7654817406620488)]


La columna  FLOW_DURATION_MILLISECONDS  tuvo correlación con las columnas
[('FLOW_DURATION_MICROSECONDS', 0.999999999972291), ('IN_BYTES', 0.7654161809786645), ('IN_PKTS', 0.7951979134524083), ('OUT_PKTS

In [68]:
# Remove useless columns
columns_to_drop = [
    "DST_TO_SRC_SECOND_BYTES", "FLOW_END_MILLISECONDS", "FLOW_END_SEC", "FLOW_START_MILLISECONDS", "FLOW_START_SEC",
    "FLOW_DURATION_MICROSECONDS",
    "IN_BYTES",
    "TCP_WIN_MIN_OUT",
    "TCP_WIN_MIN_IN",
    "TCP_WIN_MAX_IN"
]
df = df.drop(columns=columns_to_drop, axis=1)

### Categorical variables preprocessing

#### Create PROTOCOL_MAP dummies

In [69]:
# Create the dummies
df = pd.get_dummies(df, columns=["PROTOCOL_MAP"])
df.head(3)

,FIRST_SWITCHED,FLOW_DURATION_MILLISECONDS,IN_PKTS,L4_DST_PORT,L4_SRC_PORT,LAST_SWITCHED,OUT_BYTES,OUT_PKTS,SRC_TO_DST_SECOND_BYTES,TCP_FLAGS,TCP_WIN_MAX_OUT,TCP_WIN_MSS_IN,L7_PROTO_NAME,LABEL,PROTOCOL_MAP_icmp,PROTOCOL_MAP_ipv6-icmp,PROTOCOL_MAP_tcp,PROTOCOL_MAP_udp
3369345,1618227349,110802,2.639057,80,49513,1618227459,6.436150,2.484907,7.150701,24,114,0,HTTP,Denial of Service R-U-Dead-Yet,0,0,1,0
4767061,1618233245,10789,1.609438,80,53526,1618233256,5.049856,1.098612,5.855072,25,114,0,HTTP,Denial of Service R-U-Dead-Yet,0,0,1,0
2811993,1618225028,114012,2.708050,80,21791,1618225142,6.601230,2.639057,7.354362,26,27960,1410,HTTP,Denial of Service R-U-Dead-Yet,0,0,1,0


#### L7_PROTO_NAME dummies

In [70]:
df["L7_PROTO_NAME"].unique()

array(['HTTP', 'HTTP.TargusDataspeed', 'TLS', 'SSH', 'DNS',
       'TLS.Microsoft', 'Unknown', 'TLS.Skype', 'TLS.Amazon',
       'BitTorrent', 'SMBv23', 'TLS.Google', 'ICMP', 'DNS.Google',
       'DNS.Microsoft', 'DNS.Amazon', 'STUN.Messenger', 'DNS.Yahoo',
       'ICMP.Amazon', 'Google', 'TLS.Facebook', 'SOCKS', 'DNS.Spotify',
       'NFS', 'TLS.Teams', 'DNS.Telegram', 'NTP', 'TLS.Microsoft365',
       'DNS.WhatsApp', 'DNS.Microsoft365', 'TLS.Cloudflare',
       'TLS.Dropbox', 'DNS.Facebook', 'TLS.GoogleServices',
       'HTTP.Microsoft', 'TeamViewer', 'QUIC.Google', 'Playstation',
       'HTTP_Proxy', 'DNS.GoogleServices', 'TLS.Apple', 'OpenVPN',
       'DNS.AnyDesk', 'TLS.YouTube', 'DNS.Apple', 'HTTP.Google',
       'TLS.Tumblr', 'DNS.Cloudflare', 'Telnet', 'STUN.SkypeCall',
       'DNS.Dropbox', 'DNS.UbuntuONE', 'TLS.TeamViewer',
       'STUN.GoogleHangoutDuo', 'QUIC.Instagram', 'HTTP_Proxy.HTTP',
       'NTP.Cloudflare', 'HTTP.TeamViewer', 'Steam', 'DNS.Teams',
       'Amazon', 'D

In [71]:
df["L7_PROTO_NAME"] = df["L7_PROTO_NAME"].map(lambda protocol: utils.lematize_protocol(protocol).upper())

In [72]:
print(df["L7_PROTO_NAME"].value_counts())

proto_name_dummies = df["L7_PROTO_NAME"].unique()

HTTP           241062
UNKNOWN        154939
DNS             35131
TLS             31883
ICMP             8795
                ...  
MS_ONEDRIVE         1
WHATSAPP            1
MEGACO              1
OPENDNS             1
ZABBIX              1
Name: L7_PROTO_NAME, Length: 107, dtype: int64


In [73]:
df = pd.get_dummies(df, columns = ["L7_PROTO_NAME"])
df.head(3)

,FIRST_SWITCHED,FLOW_DURATION_MILLISECONDS,IN_PKTS,L4_DST_PORT,L4_SRC_PORT,LAST_SWITCHED,OUT_BYTES,OUT_PKTS,SRC_TO_DST_SECOND_BYTES,TCP_FLAGS,...,L7_PROTO_NAME_VMWARE,L7_PROTO_NAME_VNC,L7_PROTO_NAME_WHATSAPP,L7_PROTO_NAME_WHATSAPPFILES,L7_PROTO_NAME_WHOIS-DAS,L7_PROTO_NAME_WIREGUARD,L7_PROTO_NAME_WSD,L7_PROTO_NAME_XBOX,L7_PROTO_NAME_XDMCP,L7_PROTO_NAME_ZABBIX
3369345,1618227349,110802,2.639057,80,49513,1618227459,6.436150,2.484907,7.150701,24,...,0,0,0,0,0,0,0,0,0,0
4767061,1618233245,10789,1.609438,80,53526,1618233256,5.049856,1.098612,5.855072,25,...,0,0,0,0,0,0,0,0,0,0
2811993,1618225028,114012,2.708050,80,21791,1618225142,6.601230,2.639057,7.354362,26,...,0,0,0,0,0,0,0,0,0,0


##### Remove PROTO_NAME dummies with less than 200 occurences

In [74]:
count = 0
for name in proto_name_dummies:
    column = f"L7_PROTO_NAME_{name}"
    if len(df[df[column] == 1]) < 200:
        df = df.drop(column, axis=1)
        print(f"Removed column: {column}")
        count += 1

print(f"Removed {count} columns.")


Removed column: L7_PROTO_NAME_SMBV23
Removed column: L7_PROTO_NAME_SOCKS
Removed column: L7_PROTO_NAME_PLAYSTATION
Removed column: L7_PROTO_NAME_OPENVPN
Removed column: L7_PROTO_NAME_TELNET
Removed column: L7_PROTO_NAME_STEAM
Removed column: L7_PROTO_NAME_MSSQL-TDS
Removed column: L7_PROTO_NAME_RDP
Removed column: L7_PROTO_NAME_MEMCACHED
Removed column: L7_PROTO_NAME_WHATSAPPFILES
Removed column: L7_PROTO_NAME_SNMP
Removed column: L7_PROTO_NAME_IMAPS
Removed column: L7_PROTO_NAME_LDAP
Removed column: L7_PROTO_NAME_SIP
Removed column: L7_PROTO_NAME_POSTGRESQL
Removed column: L7_PROTO_NAME_CLOUDFLARE
Removed column: L7_PROTO_NAME_REDIS
Removed column: L7_PROTO_NAME_IMAP
Removed column: L7_PROTO_NAME_SSDP
Removed column: L7_PROTO_NAME_SKYPE
Removed column: L7_PROTO_NAME_WSD
Removed column: L7_PROTO_NAME_APPLE
Removed column: L7_PROTO_NAME_RTP
Removed column: L7_PROTO_NAME_VNC
Removed column: L7_PROTO_NAME_S7COMM
Removed column: L7_PROTO_NAME_FTP_CONTROL
Removed column: L7_PROTO_NAME_DNP3


In [75]:
df.shape

(488370, 32)

#### Treat LABEL column

In [76]:
df["LABEL"].unique()

array(['Denial of Service R-U-Dead-Yet', 'Denial of Service Slowloris',
       'Normal flow', 'SYN Scan - aggressive'], dtype=object)

In [77]:
labels_dic = {
    "Normal flow": 0,
    "SYN Scan - aggressive": 1,
    "Denial of Service R-U-Dead-Yet": 2,
    "Denial of Service Slowloris": 3
}
df["LABEL"] = df["LABEL"].apply(lambda x: labels_dic[x])
df["LABEL"].unique()

array([2, 3, 0, 1])

### Normalize column names

In [78]:
# Normalize columns
df.columns = [column.upper().replace("-", "_") for column in df.columns]
df.columns

Index(['FIRST_SWITCHED', 'FLOW_DURATION_MILLISECONDS', 'IN_PKTS',
       'L4_DST_PORT', 'L4_SRC_PORT', 'LAST_SWITCHED', 'OUT_BYTES', 'OUT_PKTS',
       'SRC_TO_DST_SECOND_BYTES', 'TCP_FLAGS', 'TCP_WIN_MAX_OUT',
       'TCP_WIN_MSS_IN', 'LABEL', 'PROTOCOL_MAP_ICMP',
       'PROTOCOL_MAP_IPV6_ICMP', 'PROTOCOL_MAP_TCP', 'PROTOCOL_MAP_UDP',
       'L7_PROTO_NAME_AMAZON', 'L7_PROTO_NAME_BITTORRENT', 'L7_PROTO_NAME_DNS',
       'L7_PROTO_NAME_GOOGLE', 'L7_PROTO_NAME_HTTP',
       'L7_PROTO_NAME_HTTP_PROXY', 'L7_PROTO_NAME_ICMP', 'L7_PROTO_NAME_NFS',
       'L7_PROTO_NAME_NTP', 'L7_PROTO_NAME_QUIC', 'L7_PROTO_NAME_SSH',
       'L7_PROTO_NAME_STUN', 'L7_PROTO_NAME_TEAMVIEWER', 'L7_PROTO_NAME_TLS',
       'L7_PROTO_NAME_UNKNOWN'],
      dtype='object')

### Third profile report

In [79]:
df.shape

(488370, 32)

In [80]:
if not os.path.isfile("reports/profile-categorical.html") and generate_reports:
    ProfileReport(df, title="Exploratory Data Analysis 2").to_file("reports/profile-categorical.html")

### Create clean dataset

In [81]:
df.to_csv("dataset/clean_df.csv", index_label=False)

In [86]:
print(len(df.columns))

df.columns

32


Index(['FIRST_SWITCHED', 'FLOW_DURATION_MILLISECONDS', 'IN_PKTS',
       'L4_DST_PORT', 'L4_SRC_PORT', 'LAST_SWITCHED', 'OUT_BYTES', 'OUT_PKTS',
       'SRC_TO_DST_SECOND_BYTES', 'TCP_FLAGS', 'TCP_WIN_MAX_OUT',
       'TCP_WIN_MSS_IN', 'LABEL', 'PROTOCOL_MAP_ICMP',
       'PROTOCOL_MAP_IPV6_ICMP', 'PROTOCOL_MAP_TCP', 'PROTOCOL_MAP_UDP',
       'L7_PROTO_NAME_AMAZON', 'L7_PROTO_NAME_BITTORRENT', 'L7_PROTO_NAME_DNS',
       'L7_PROTO_NAME_GOOGLE', 'L7_PROTO_NAME_HTTP',
       'L7_PROTO_NAME_HTTP_PROXY', 'L7_PROTO_NAME_ICMP', 'L7_PROTO_NAME_NFS',
       'L7_PROTO_NAME_NTP', 'L7_PROTO_NAME_QUIC', 'L7_PROTO_NAME_SSH',
       'L7_PROTO_NAME_STUN', 'L7_PROTO_NAME_TEAMVIEWER', 'L7_PROTO_NAME_TLS',
       'L7_PROTO_NAME_UNKNOWN'],
      dtype='object')